In [ ]:
%load_ext rpy2.ipython
from datetime import datetime, timedelta
import pandas as pd
from rpy2.robjects import r, pandas2ri
import numpy as np

pandas2ri.activate()

In [ ]:
%%R
library("depmixS4")

### Import dataset

In [ ]:
df = pd.read_csv('/directory/file_name.csv')

#### Bin the channels - categories from 1

In [ ]:
df['Outgoing call'] = df['Outgoing call'].apply(lambda x: 2 if x>0 else 1)
df['Incoming call'] = df['Incoming call'].apply(lambda x: 2 if x>0 else 1)
df['Social media App'] = df['Social media App'].apply(lambda x: 2 if x>0 else 1)
df['Communication App'] = df['Communication App'].apply(lambda x: 2 if x>0 else 1)
df['Phone Usage'] = df['Phone Usage'].apply(lambda x: 2 if x>0 else 1)

#### Missing data

In [ ]:
#  If has_data is 0, set all other columns to NaN
df.loc[df['has_data'] == 0, 'Outgoing call'] = np.nan
df.loc[df['has_data'] == 0, 'Incoming call'] = np.nan
df.loc[df['has_data'] == 0, 'Social media App'] = np.nan
df.loc[df['has_data'] == 0, 'Communication App'] = np.nan
df.loc[df['has_data'] == 0, 'Phone Usage'] = np.nan

### Group model training

In [ ]:
# Encode hour using onehot encoding
df = df.sort_values(by=['Hour'])  # sorting to make sure hours always get same assignment regardless of which is the first value in the df
onehot_hour = pd.get_dummies(df['Hour'], dtype=float, prefix="onehot_hour", drop_first=True)
df = pd.concat([df, onehot_hour], axis=1)
df = df.sort_index() 

In [ ]:
# For training/testing split:
df_training = df[df['Testing1_Training0']==0]
df_subset = df_training[['ID', 'Hour', 'Outgoing call', 'Incoming call', 'Social media App', 'Communication App', 'Phone Usage','onehot_hour_1',
       'onehot_hour_2', 'onehot_hour_3', 'onehot_hour_4', 'onehot_hour_5',
       'onehot_hour_6', 'onehot_hour_7', 'onehot_hour_8', 'onehot_hour_9',
       'onehot_hour_10', 'onehot_hour_11', 'onehot_hour_12', 'onehot_hour_13',
       'onehot_hour_14', 'onehot_hour_15', 'onehot_hour_16', 'onehot_hour_17',
       'onehot_hour_18', 'onehot_hour_19', 'onehot_hour_20', 'onehot_hour_21',
       'onehot_hour_22', 'onehot_hour_23']]

In [ ]:
# Get vector containing lengths of time series for each training participant
IDs = list(df_subset['ID'].unique())
sequence_length = []
for ID in IDs:
    length = len(df_subset[df_subset['ID']==ID])
    sequence_length.append(length)

In [ ]:
%%R -i df_subset,sequence_length
# Train model
df_subset[df_subset == 'NaN'] <- NA

start_time <- Sys.time()
colnames(df_subset) <- c('ID', 'Hour', 'Outgoing.call', 'Incoming.call', 'Social.media.App', 'Communication.App', 'Phone.Usage','onehot_hour_1','onehot_hour_2', 'onehot_hour_3', 'onehot_hour_4', 'onehot_hour_5', 'onehot_hour_6', 'onehot_hour_7', 'onehot_hour_8', 'onehot_hour_9', 'onehot_hour_10', 'onehot_hour_11', 'onehot_hour_12', 'onehot_hour_13', 'onehot_hour_14', 'onehot_hour_15', 'onehot_hour_16', 'onehot_hour_17', 'onehot_hour_18', 'onehot_hour_19', 'onehot_hour_20', 'onehot_hour_21', 'onehot_hour_22', 'onehot_hour_23')
n_states <- 2
mod <- depmix(list(Outgoing.call ~ 1 
                   , Incoming.call ~ 1
                   , Social.media.App ~ 1
                   , Communication.App ~ 1
                   , Phone.Usage ~ 1
                  ), data = df_subset, nstates = n_states,
 family = list(multinomial(link="identity")
               , multinomial(link="identity"), multinomial(link="identity")
               , multinomial(link="identity"), multinomial(link="identity")
              ),
              ntimes=as.vector(sequence_length)
              , transition=~onehot_hour_1+onehot_hour_2+ onehot_hour_3+ onehot_hour_4+ onehot_hour_5+onehot_hour_6+ onehot_hour_7+ onehot_hour_8+ onehot_hour_9+onehot_hour_10+ onehot_hour_11+ onehot_hour_12+ onehot_hour_13+onehot_hour_14+ onehot_hour_15+ onehot_hour_16+ onehot_hour_17+onehot_hour_18+ onehot_hour_19+ onehot_hour_20+ onehot_hour_21+onehot_hour_22+ onehot_hour_23  # time-varying covariates for transition probabilities
             )

fm <- fit(mod)
end_time <- Sys.time()
print(end_time - start_time)

filename = paste('/directory/file_name_', as.character(n_states), '.rda', sep='')
saveRDS(fm, file =filename)


### Quantifications

In [ ]:
%%R 
fm <- readRDS('/directory/file_name.rda')
print(logLik(fm))
print(summary(fm))

### Predict hidden state sequence for training segments (participants)

In [ ]:
%%R
pst_viterbi <- posterior(fm,type="viterbi")
group_state_sequence <- pst_viterbi['state']

In [ ]:
# Save hidden state sequence
training_overall_state_sequence = r['group_state_sequence']
df_training_overall_state_sequence = pd.DataFrame(columns=['State'])
df_training_overall_state_sequence['State'] = list(training_overall_state_sequence)[0]
df_training_overall_state_sequence.to_csv('/directory/file_name.csv', index=False)

In [ ]:
# If loading hidden state sequence
df_training_overall_state_sequence = pd.read_csv('/directory/file_name.csv')
df_training_overall_state_sequence.rename(columns={"State": "state"}, inplace=True)

In [ ]:
# Drop the periods where data was missing from df_training_overall_state_sequence
# Combine the state sequence in df with rest of data, drop has_data=0 rows, then take out state sequence column so can get new length per p
df_training_overall_state_sequence['has_data'] = df_training['has_data']
df_training_overall_state_sequence['ID'] = df_training['ID']
df_training_overall_state_sequence = df_training_overall_state_sequence.loc[df_training_overall_state_sequence['has_data']==1]

# Also calculate remaining lengths for sequences
IDs = list(df_training_overall_state_sequence['ID'].unique())
sequence_length_training = []
for ID in IDs:
    length = len(df_training_overall_state_sequence[df_training_overall_state_sequence['ID']==ID])
    sequence_length_training.append(length)
    
df_training_overall_state_sequence = df_training_overall_state_sequence.drop(['has_data','ID'], axis=1)

In [ ]:
%%R -i df_training_overall_state_sequence
group_state_sequence <- df_training_overall_state_sequence

#### Training segment (participant) dwell times

In [ ]:
%%R -i sequence_length_training
all_training_segment_dwell_times <- list()
start <- 1
for (s in sequence_length_training)
    {
    end <- start + s - 1
    segment_state_sequence <- group_state_sequence[start:end,1]    
    dwell_time_state1 <- 100 * length(which(segment_state_sequence==1))/length(segment_state_sequence)
    dwell_time_state2 <- 100 * length(which(segment_state_sequence==2))/length(segment_state_sequence)
    segment_dwell_times <- list(dwell_time_state1,dwell_time_state2)#,dwell_time_state3,dwell_time_state4)
    print(segment_dwell_times)
    all_training_segment_dwell_times <- append(all_training_segment_dwell_times,segment_dwell_times) 
    start <- end + 1
}

In [ ]:
all_training_segment_dwell_times = r['all_training_segment_dwell_times']

In [ ]:
dwell_times_state1 = all_training_segment_dwell_times[0:len(all_training_segment_dwell_times):2]
dwell_times_state2 = all_training_segment_dwell_times[1:len(all_training_segment_dwell_times):2]
dwell_times_state1 = [dwell_times_state1[i][0] for i in range(len(dwell_times_state1))]
dwell_times_state2 = [dwell_times_state2[i][0] for i in range(len(dwell_times_state2))]

In [ ]:
df_dwell_times = pd.DataFrame(columns=['ID', 'SFS', 'LONELINESS', 'AGE', 'LABEL', 'DATASET', 'dwell_times_state1','dwell_times_state2', 'Testing1_Training0'])
df_training_ID_unique = df_training.drop_duplicates(subset=['ID'])
df_dwell_times['ID'] = df_training_ID_unique['ID']
df_dwell_times['SFS'] = df_training_ID_unique['SFS']
df_dwell_times['LONELINESS'] = df_training_ID_unique['LONELINESS']
df_dwell_times['AGE'] = df_training_ID_unique['AGE']
df_dwell_times['LABEL'] = df_training_ID_unique['LABEL']
df_dwell_times['DATASET'] = df_training_ID_unique['DATASET']
df_dwell_times['dwell_times_state1'] = dwell_times_state1
df_dwell_times['dwell_times_state2'] = dwell_times_state2
df_dwell_times['Testing1_Training0'] = 0
df_dwell_times.to_csv('/directory/file_name.csv', index=False)

### Predict hidden state sequence for testing segments (participants)

In [ ]:
df_testing = df[(df['Testing1_Training0']==1)]

In [ ]:
# Get vector containing lengths of time series for each participant
df_testing = df_testing[['ID', 'Hour', 'has_data', 'Outgoing call', 'Incoming call', 'Social media App', 'Communication App', 'Phone Usage','onehot_hour_1',
       'onehot_hour_2', 'onehot_hour_3', 'onehot_hour_4', 'onehot_hour_5',
       'onehot_hour_6', 'onehot_hour_7', 'onehot_hour_8', 'onehot_hour_9',
       'onehot_hour_10', 'onehot_hour_11', 'onehot_hour_12', 'onehot_hour_13',
       'onehot_hour_14', 'onehot_hour_15', 'onehot_hour_16', 'onehot_hour_17',
       'onehot_hour_18', 'onehot_hour_19', 'onehot_hour_20', 'onehot_hour_21',
       'onehot_hour_22', 'onehot_hour_23']]
sequence_length_testing = []
for ID in list(df_testing['ID'].unique()):
    length = len(df_testing[df_testing['ID']==ID])
    sequence_length_testing.append(length)

In [ ]:
# %%R
# initial_probabilities <- getpars(fm)[1:2]
# transition_matrix <- getpars(fm)[3:6]
# emission_probabilities <- getpars(fm)[7:36]

In [ ]:
%%R -i df_testing,sequence_length_testing
df_testing[df_testing == 'NaN'] <- NA
colnames(df_testing) <- c('ID', 'Hour', 'has_data', 'Outgoing.call', 'Incoming.call', 'Social.media.App', 'Communication.App', 'Phone.Usage','onehot_hour_1','onehot_hour_2', 'onehot_hour_3', 'onehot_hour_4', 'onehot_hour_5',        'onehot_hour_6', 'onehot_hour_7', 'onehot_hour_8', 'onehot_hour_9',        'onehot_hour_10', 'onehot_hour_11', 'onehot_hour_12', 'onehot_hour_13',        'onehot_hour_14', 'onehot_hour_15', 'onehot_hour_16', 'onehot_hour_17',        'onehot_hour_18', 'onehot_hour_19', 'onehot_hour_20', 'onehot_hour_21',        'onehot_hour_22', 'onehot_hour_23')
n_states <- 2
mod_testing <- depmix(list(Outgoing.call ~ 1
                   , Incoming.call ~ 1
                   , Social.media.App ~ 1
                   , Communication.App ~ 1
                   , Phone.Usage ~ 1
                  ), data = df_testing, nstates = n_states,
 family = list(multinomial(link="identity")
               , multinomial(link="identity"), multinomial(link="identity")
               , multinomial(link="identity"), multinomial(link="identity")
              ),
              ntimes=as.vector(sequence_length_testing)
              , transition= ~ onehot_hour_1+onehot_hour_2+ onehot_hour_3+ onehot_hour_4+ onehot_hour_5+onehot_hour_6+ onehot_hour_7+ onehot_hour_8+ onehot_hour_9+onehot_hour_10+ onehot_hour_11+ onehot_hour_12+ onehot_hour_13+onehot_hour_14+ onehot_hour_15+ onehot_hour_16+ onehot_hour_17+onehot_hour_18+ onehot_hour_19+ onehot_hour_20+ onehot_hour_21+onehot_hour_22+ onehot_hour_23  # time-varying covariates # runs with ~Hour but need transform
             )
mod_testing <- setpars(mod_testing,getpars(fm))  # Use parameters from trained model

In [ ]:
%%R
pst_viterbi <- posterior(mod_testing,type="viterbi")
testing_overall_state_sequence <- pst_viterbi['state']

In [ ]:
# Save hidden state sequence
testing_overall_state_sequence = r['testing_overall_state_sequence']
testing_overall_state_sequence = testing_overall_state_sequence[:]
df_testing_overall_state_sequence = pd.DataFrame(columns=['State'])
df_testing_overall_state_sequence['State'] = list(testing_overall_state_sequence)[0][:-1]
df_testing_overall_state_sequence.to_csv('/directory/file_name.csv', index=False)

In [ ]:
# Drop the periods where data was missing from df_training_overall_state_sequence
# Combine the state sequence in df with rest of data, drop has_data=0 rows, then take out state sequence column so can get new length per p
df_testing_overall_state_sequence['has_data'] = list(df_testing['has_data'])
df_testing_overall_state_sequence['ID'] = list(df_testing['ID'])
print(len(df_testing['ID'].unique()))
df_testing_overall_state_sequence = df_testing_overall_state_sequence.loc[df_testing_overall_state_sequence['has_data']==1]

# Also calculate remaining lengths for sequences
IDs = list(df_testing_overall_state_sequence['ID'].unique())
print(len(IDs))
sequence_length_testing = []
for ID in IDs:
    length = len(df_testing_overall_state_sequence[df_testing_overall_state_sequence['ID']==ID])
    sequence_length_testing.append(length)
    
df_testing_overall_state_sequence = df_testing_overall_state_sequence.drop(['has_data','ID'], axis=1)

In [ ]:
%%R -i sequence_length_testing
all_testing_segment_dwell_times <- list()
start <- 1
for (s in sequence_length_testing)
    {
    end <- start + s - 1
    testing_state_sequence <- testing_overall_state_sequence[start:end,1]    
    dwell_time_state1 <- 100 * length(which(testing_state_sequence==1))/length(testing_state_sequence)
    dwell_time_state2 <- 100 * length(which(testing_state_sequence==2))/length(testing_state_sequence)
    segment_dwell_times <- list(dwell_time_state1,dwell_time_state2)#,dwell_time_state3,dwell_time_state4)
    print(segment_dwell_times)
    all_testing_segment_dwell_times <- append(all_testing_segment_dwell_times,segment_dwell_times) 
    start <- end + 1
    
}

In [ ]:
all_testing_segment_dwell_times = r['all_testing_segment_dwell_times']

In [ ]:
dwell_times_state1 = all_testing_segment_dwell_times[0:len(all_testing_segment_dwell_times):2]
dwell_times_state2 = all_testing_segment_dwell_times[1:len(all_testing_segment_dwell_times):2]
dwell_times_state1 = [dwell_times_state1[i][0] for i in range(len(dwell_times_state1))]
dwell_times_state2 = [dwell_times_state2[i][0] for i in range(len(dwell_times_state2))]

In [ ]:
df_testing = df[(df['Testing1_Training0']==1)]

In [ ]:
df_dwell_times = pd.DataFrame(columns=['ID', 'SFS', 'LONELINESS', 'LABEL', 'AGE', 'SEX', 'RACE',
       'EDUCATION YEARS', 'COUNTRY', 'DATASET', 'dwell_times_state1', 'dwell_times_state2', 'Testing1_Training0'])
df_testing_ID_unique = df_testing.drop_duplicates(subset=['ID'])
df_dwell_times['ID'] = df_testing_ID_unique['ID']
df_dwell_times['SFS'] = df_testing_ID_unique['SFS']
df_dwell_times['LONELINESS'] = df_testing_ID_unique['LONELINESS']
df_dwell_times['LABEL'] = df_testing_ID_unique['LABEL']
df_dwell_times['AGE'] = df_testing_ID_unique['AGE']
df_dwell_times['SEX'] = df_testing_ID_unique['SEX']
df_dwell_times['RACE'] = df_testing_ID_unique['RACE']
df_dwell_times['EDUCATION YEARS'] = df_testing_ID_unique['EDUCATION YEARS']
df_dwell_times['COUNTRY'] = df_testing_ID_unique['COUNTRY']
df_dwell_times['DATASET'] = df_testing_ID_unique['DATASET']
df_dwell_times['dwell_times_state1'] = dwell_times_state1
df_dwell_times['dwell_times_state2'] = dwell_times_state2
df_dwell_times['Testing1_Training0'] = 1
df_dwell_times.to_csv('/directory/file_name.csv', index=False)

### Inclusion of individual transition coefficients

If individual transition coefficients were included in the model (i.e. +ID in the transition formula) then the coefficients can be converted to probabilities using this function, where the mean probability is calculated across the day:

In [ ]:
%%R
convert_coefficients_to_probabilities <- function(from_state_n_intercepts,from_state_n_id_coefficients_matrix, from_state_n_hour_coefficients_matrix){
    #To get probabilities for each person per hour and then do mean
    # from_state_n_intercepts contains the intercepts, from_state_n_id_coefficients_matrix contains the coefficients for each ID, and from_state_n_hour_coefficients_matrix contains the coefficients for each hour
    #Empty vector for probs for each hour
    into_state_1_probabilities <- c()
    into_state_2_probabilities <- c()
    into_state_3_probabilities <- c()
    into_state_4_probabilities <- c()
    into_state_5_probabilities <- c()
    # Add intercepts to each value 
    from_state_n_added_intercept_to_each_coefficient <- from_state_n_id_coefficients_matrix + rep(from_state_n_intercepts, each = nrow(from_state_n_id_coefficients_matrix))
    # Add back in intercept row as use this for first participant
    from_state_n_added_intercept_to_each_coefficient <- rbind(from_state_n_intercepts, from_state_n_added_intercept_to_each_coefficient)
    
    # Do probs for hour zero:
    # Exponentiate
    from_state_n_added_intercept_to_each_coefficient_exp <- exp(from_state_n_added_intercept_to_each_coefficient)
    # Make denominator (sum each row, don't do +1 as exp'ing 0 gives the +1)
    from_state_n_added_intercept_to_each_coefficient_denominator <- apply(from_state_n_added_intercept_to_each_coefficient_exp, 1, sum) # the '1,sum' bit means sum across each row
    # Divide by denominator
    trans_probs <- from_state_n_added_intercept_to_each_coefficient_exp/from_state_n_added_intercept_to_each_coefficient_denominator  
    into_state_1_probabilities <- cbind(into_state_1_probabilities, trans_probs[,1])
    into_state_2_probabilities <- cbind(into_state_2_probabilities, trans_probs[,2])
    into_state_3_probabilities <- cbind(into_state_3_probabilities, trans_probs[,3])
    into_state_4_probabilities <- cbind(into_state_4_probabilities, trans_probs[,4])
    into_state_5_probabilities <- cbind(into_state_5_probabilities, trans_probs[,5])   
    # Then need to add in coefficient for all other hours
    for (i in 1:23) {
        # Get hour coefficients
        from_state_n_hour_coefficient <- from_state_n_hour_coefficients_matrix[i,]
        # Add hour coefficients to the ID+intercept matrix
        from_state_n_added_hour_to_each_coefficient <- from_state_n_added_intercept_to_each_coefficient + rep(from_state_n_hour_coefficient, each = nrow(from_state_n_added_intercept_to_each_coefficient))
        # For transitions into state 1 need to do differently. So set 1st column back to 0 so it will still give 1 when exponentiate (and therefore provide correct value in denominator)
        # Exponentiate
        from_state_n_added_hour_to_each_coefficient_exp <- exp(from_state_n_added_hour_to_each_coefficient)
        # Make denominator 
        from_state_n_added_hour_to_each_coefficient_denominator <- apply(from_state_n_added_hour_to_each_coefficient_exp, 1, sum) # the '1,sum' bit means sum across each row
        # Divide by denominator
        trans_probs <- from_state_n_added_hour_to_each_coefficient_exp/from_state_n_added_hour_to_each_coefficient_denominator  
        # Store each probability
        into_state_1_probabilities <- cbind(into_state_1_probabilities, trans_probs[,1])
        into_state_2_probabilities <- cbind(into_state_2_probabilities, trans_probs[,2])
        into_state_3_probabilities <- cbind(into_state_3_probabilities, trans_probs[,3])
        into_state_4_probabilities <- cbind(into_state_4_probabilities, trans_probs[,4])
        into_state_5_probabilities <- cbind(into_state_5_probabilities, trans_probs[,5])
    }
    #Calculate means
    into_state_1_probabilities_mean <- apply(into_state_1_probabilities, 1, mean)
    into_state_2_probabilities_mean <- apply(into_state_2_probabilities, 1, mean)
    into_state_3_probabilities_mean <- apply(into_state_3_probabilities, 1, mean)
    into_state_4_probabilities_mean <- apply(into_state_4_probabilities, 1, mean)
    into_state_5_probabilities_mean <- apply(into_state_5_probabilities, 1, mean)

    mean_trans_probs <- cbind(into_state_1_probabilities_mean, into_state_2_probabilities_mean, into_state_3_probabilities_mean, into_state_4_probabilities_mean, into_state_5_probabilities_mean)

    return(mean_trans_probs)
}
